In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

**Naive Data Representation**

In [3]:
x1_train = torch.FloatTensor([[73], [93], [89], [96], [73]])
x2_train = torch.FloatTensor([[80], [88], [91], [98], [66]])
x3_train = torch.FloatTensor([[75], [93], [90], [100], [70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [4]:
w1 = torch.zeros(1, requires_grad = True)
w2 = torch.zeros(1, requires_grad = True)
w3 = torch.zeros(1, requires_grad = True)
b = torch.zeros(1, requires_grad = True)

optimizer = optim.SGD([w1, w2, w3, b], lr = 1e-5)

nb_epochs = 1000

for epoch in range(nb_epochs + 1):
    hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b
    cost = torch.mean((hypothesis - y_train) ** 2)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{}, w1: {:.3f}, w2: {:.3f}, w3: {:.3f}, b: {:.3f}, Cost: {:.6f}'.format(epoch, nb_epochs, w1.item(), w3.item(), w3.item(), b.item(), cost.item()))

Epoch    0/1000, w1: 0.294, w2: 0.297, w3: 0.297, b: 0.003, Cost: 29661.800781
Epoch  100/1000, w1: 0.674, w2: 0.676, w3: 0.676, b: 0.008, Cost: 1.563628
Epoch  200/1000, w1: 0.679, w2: 0.677, w3: 0.677, b: 0.008, Cost: 1.497595
Epoch  300/1000, w1: 0.684, w2: 0.677, w3: 0.677, b: 0.008, Cost: 1.435044
Epoch  400/1000, w1: 0.689, w2: 0.678, w3: 0.678, b: 0.008, Cost: 1.375726
Epoch  500/1000, w1: 0.694, w2: 0.678, w3: 0.678, b: 0.009, Cost: 1.319507
Epoch  600/1000, w1: 0.699, w2: 0.679, w3: 0.679, b: 0.009, Cost: 1.266222
Epoch  700/1000, w1: 0.704, w2: 0.679, w3: 0.679, b: 0.009, Cost: 1.215703
Epoch  800/1000, w1: 0.709, w2: 0.679, w3: 0.679, b: 0.009, Cost: 1.167810
Epoch  900/1000, w1: 0.713, w2: 0.680, w3: 0.680, b: 0.009, Cost: 1.122429
Epoch 1000/1000, w1: 0.718, w2: 0.680, w3: 0.680, b: 0.009, Cost: 1.079390


**Matrix Data Representation**

In [5]:
x_train = torch.FloatTensor([[73, 80, 75], [93, 88, 93], [89, 91, 90], [96, 98, 100], [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [7]:
print(x_train.shape)
print(y_train.shape)

torch.Size([5, 3])
torch.Size([5, 1])


In [9]:
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 20

for epoch in range(nb_epochs + 1):
    hypothesis = x_train.matmul(W) + b # or .mm or @
    cost = torch.mean((hypothesis - y_train) ** 2)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{}, hypothesis: {}, Cost: {:.6f}'.format(epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()))

Epoch    0/20, hypothesis: tensor([0., 0., 0., 0., 0.]), Cost: 29661.800781
Epoch    1/20, hypothesis: tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605]), Cost: 9298.520508
Epoch    2/20, hypothesis: tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821]), Cost: 2915.712402
Epoch    3/20, hypothesis: tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097]), Cost: 915.040527
Epoch    4/20, hypothesis: tensor([137.7967, 165.6247, 163.1911, 177.7112, 126.3307]), Cost: 287.936096
Epoch    5/20, hypothesis: tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891]), Cost: 91.371063
Epoch    6/20, hypothesis: tensor([148.1035, 178.0143, 175.3980, 191.0042, 135.7812]), Cost: 29.758249
Epoch    7/20, hypothesis: tensor([150.1744, 180.5042, 177.8509, 193.6753, 137.6805]), Cost: 10.445267
Epoch    8/20, hypothesis: tensor([151.3336, 181.8983, 179.2240, 195.1707, 138.7440]), Cost: 4.391237
Epoch    9/20, hypothesis: tensor([151.9824, 182.6789, 179.9928, 196.0079, 139.3396]), Cost: 2.49312

**High level Implementation with nn.Module**

In [10]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

In [12]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

In [13]:
x_train = torch.FloatTensor([[73, 80, 75], [93, 88, 93], [89, 91, 90], [96, 98, 100], [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

model = MultivariateLinearRegressionModel()
optimizer = optim.SGD(model.parameters(), lr = 1e-5)

nb_epochs = 20

for epoch in range(nb_epochs+1):
    prediction = model(x_train)
    cost = F.mse_loss(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    print('Epoch {:4d}/{}, Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/20, Cost: 31667.597656
Epoch    1/20, Cost: 9926.265625
Epoch    2/20, Cost: 3111.513672
Epoch    3/20, Cost: 975.451477
Epoch    4/20, Cost: 305.908630
Epoch    5/20, Cost: 96.042679
Epoch    6/20, Cost: 30.260782
Epoch    7/20, Cost: 9.641681
Epoch    8/20, Cost: 3.178685
Epoch    9/20, Cost: 1.152871
Epoch   10/20, Cost: 0.517862
Epoch   11/20, Cost: 0.318802
Epoch   12/20, Cost: 0.256388
Epoch   13/20, Cost: 0.236810
Epoch   14/20, Cost: 0.230660
Epoch   15/20, Cost: 0.228719
Epoch   16/20, Cost: 0.228095
Epoch   17/20, Cost: 0.227880
Epoch   18/20, Cost: 0.227799
Epoch   19/20, Cost: 0.227759
Epoch   20/20, Cost: 0.227732


**Slicing 1D Array**

In [14]:
nums = [0, 1, 2, 3, 4]

print(nums)

[0, 1, 2, 3, 4]


In [15]:
print(nums[2: 4])

[2, 3]


In [16]:
print(nums[2: ])

[2, 3, 4]


In [17]:
print(nums[: 2])

[0, 1]


In [19]:
print(nums[: ])

[0, 1, 2, 3, 4]


In [20]:
print(nums[: -1])

[0, 1, 2, 3]


In [21]:
nums[2: 4] = [8, 9]

In [22]:
print(nums)

[0, 1, 8, 9, 4]


**Slicing 2D Array**

In [23]:
import numpy as np

In [26]:
b = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])

print(b)

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]


In [27]:
b[: , 1]

array([ 2,  6, 10])

In [28]:
b[-1]

array([ 9, 10, 11, 12])

In [30]:
b[-1, : ]

array([ 9, 10, 11, 12])

In [31]:
b[-1, ...]

array([ 9, 10, 11, 12])

In [32]:
b[0: 2, : ]

array([[1, 2, 3, 4],
       [5, 6, 7, 8]])

**Loading Data from .csv file**

In [33]:
import numpy as np

In [46]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [47]:
xy = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/datafile/data-01-test-score.csv', delimiter = ',', dtype = np.float32)

In [48]:
x_data = xy[: , 0: -1]
y_data = xy[: , [-1]]

In [49]:
print(x_data.shape)
print(len(x_data))
print(x_data[: 5])

(25, 3)
25
[[ 73.  80.  75.]
 [ 93.  88.  93.]
 [ 89.  91.  90.]
 [ 96.  98. 100.]
 [ 73.  66.  70.]]


In [50]:
print(y_data.shape)
print(len(y_data))
print(y_data[: 5])

(25, 1)
25
[[152.]
 [185.]
 [180.]
 [196.]
 [142.]]


**Imports**

In [52]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

**Low Level Implementation**

In [54]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

W = torch.zeros((3, 1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)
optimizer = optim.SGD([W, b], lr = 1e-5)

nb_epochs = 20

for epoch in range(nb_epochs + 1):
    hypothesis = x_train.matmul(W) + b # or .mm or @

    cost = torch.mean((hypothesis - y_train) ** 2)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{}, Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/20, Cost: 26811.960938
Epoch    1/20, Cost: 9920.530273
Epoch    2/20, Cost: 3675.298340
Epoch    3/20, Cost: 1366.260498
Epoch    4/20, Cost: 512.542480
Epoch    5/20, Cost: 196.896637
Epoch    6/20, Cost: 80.190987
Epoch    7/20, Cost: 37.038696
Epoch    8/20, Cost: 21.081343
Epoch    9/20, Cost: 15.178760
Epoch   10/20, Cost: 12.993679
Epoch   11/20, Cost: 12.183023
Epoch   12/20, Cost: 11.880535
Epoch   13/20, Cost: 11.765958
Epoch   14/20, Cost: 11.720851
Epoch   15/20, Cost: 11.701438
Epoch   16/20, Cost: 11.691514
Epoch   17/20, Cost: 11.685116
Epoch   18/20, Cost: 11.680005
Epoch   19/20, Cost: 11.675380
Epoch   20/20, Cost: 11.670952


**High Level Implementation with nn.Module**

In [55]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

In [58]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

model = MultivariateLinearRegressionModel()
optimizer = optim.SGD(model.parameters(), lr = 1e-5)

nb_epochs = 20

for epoch in range(nb_epochs + 1):
    prediction = model(x_train)

    cost = F.mse_loss(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{}, Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/20, Cost: 22444.050781
Epoch    1/20, Cost: 8304.253906
Epoch    2/20, Cost: 3076.377197
Epoch    3/20, Cost: 1143.485474
Epoch    4/20, Cost: 428.840912
Epoch    5/20, Cost: 164.614960
Epoch    6/20, Cost: 66.922005
Epoch    7/20, Cost: 30.800661
Epoch    8/20, Cost: 17.444128
Epoch    9/20, Cost: 12.504412
Epoch   10/20, Cost: 10.676523
Epoch   11/20, Cost: 9.999249
Epoch   12/20, Cost: 9.747339
Epoch   13/20, Cost: 9.652740
Epoch   14/20, Cost: 9.616273
Epoch   15/20, Cost: 9.601300
Epoch   16/20, Cost: 9.594303
Epoch   17/20, Cost: 9.590237
Epoch   18/20, Cost: 9.587271
Epoch   19/20, Cost: 9.584668
Epoch   20/20, Cost: 9.582247
